# From Judging to Synthesizing — Evolving Multi-Agent Patterns

In the original 2_lab2.ipynb, we explored a powerful agentic design pattern: sending the same question to multiple large language models (LLMs), then using a separate “judge” agent to evaluate and rank their responses. This approach is valuable for identifying the single best answer among many, leveraging the strengths of ensemble reasoning and critical evaluation.

However, selecting just one “winner” can leave valuable insights from other models untapped. To address this, I am shifting to a new agentic pattern in this notebook: the synthesizer/improver pattern. Instead of merely ranking responses, we will prompt a dedicated LLM to review all answers, extract the most compelling ideas from each, and synthesize them into a single, improved response. 

This approach aims to combine the collective intelligence of multiple models, producing an answer that is richer, more nuanced, and more robust than any individual response.


In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)

True

In [3]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)
Google API Key not set (and this is optional)
DeepSeek API Key not set (and this is optional)
Groq API Key not set (and this is optional)


In [4]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their collective intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [5]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their collective intelligence. Answer only with the question, no explanation.'}]

In [6]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


How can societies balance the ethical implications of AI advancements with the drive for innovation and economic growth, while ensuring equitable access and minimizing potential harms?


In [7]:
teammates = []
answers = []
messages = [{"role": "user", "content": question}]

In [8]:
# The API we know well

model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
teammates.append(model_name)
answers.append(answer)

Balancing the ethical implications of AI advancements with innovation and economic growth, while ensuring equitable access and minimizing potential harms, requires a multifaceted approach. Here are several strategies that societies can adopt:

1. **Framework for Ethical AI Development**:
   - Establish clear ethical guidelines and frameworks for the development and deployment of AI technologies. This includes principles such as fairness, transparency, accountability, and respect for privacy.
   - Encourage the involvement of ethicists, social scientists, and diverse stakeholders in the design and implementation of AI systems to ensure a variety of perspectives and values are considered.

2. **Inclusive Stakeholder Engagement**:
   - Create forums for public dialogue that include voices from various demographics, including marginalized communities, to ensure that AI advancements meet the needs of all citizens.
   - Involve interdisciplinary teams in AI development, capturing insights from fields such as sociology, psychology, and law to guide ethical considerations.

3. **Regulatory Frameworks and Policies**:
   - Develop effective regulations and compliance mechanisms that hold companies accountable for the ethical implications of their AI technologies without stifling innovation.
   - Implement policy measures that promote responsible AI use, such as data protection laws and guidelines for ethical AI design.

4. **Equitable Access and Inclusion**:
   - Ensure that AI technology is accessible to all segments of society, including small businesses, nonprofits, and underserved communities, to prevent widening the digital divide.
   - Promote digital literacy programs to equip individuals with the skills needed to interact with AI technologies effectively.

5. **Investment in Research and Development**:
   - Encourage investments in R&D focused on ethical AI, including projects that explore societal impacts and applications that benefit the public good.
   - Allocate funding for academic and non-profit organizations that prioritize research into the societal impacts of AI technologies and seek to develop equitable solutions.

6. **Continuous Monitoring and Adaptation**:
   - Establish mechanisms for ongoing assessment of AI systems to identify and address unintended consequences, biases, and inequities.
   - Encourage adaptive governance that can evolve with technological advancements, allowing regulations and guidelines to be updated as new ethical challenges arise.

7. **Collaboration Between Public and Private Sectors**:
   - Foster partnerships between government, academia, and industry to align interests and share best practices in responsible AI development.
   - Create initiatives that focus on ethical AI as a competitive advantage in economic growth while prioritizing societal welfare.

8. **Transparency and Accountability**:
   - Promote transparency in AI algorithms and decision-making processes to build trust and allow for scrutiny of AI systems.
   - Implement accountability measures for organizations that deploy AI, ensuring they can be held responsible for the impacts of their technologies.

9. **Public Awareness and Education**:
   - Facilitate public education campaigns that inform citizens about AI technologies, their benefits, risks, and ethical considerations.
   - Encourage discussions about the implications of AI in various sectors, allowing for informed community engagement and advocacy.

By integrating these strategies, societies can navigate the complex landscape of AI advancements, fostering innovation and economic growth while ensuring ethical considerations and equitable access are at the forefront of development.

In [ ]:
# Anthropic has a slightly different API, and Max Tokens is required

model_name = "claude-3-7-sonnet-latest"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
teammates.append(model_name)
answers.append(answer)

In [ ]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
teammates.append(model_name)
answers.append(answer)

In [ ]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
teammates.append(model_name)
answers.append(answer)

In [ ]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
teammates.append(model_name)
answers.append(answer)

In [9]:
# So where are we?

print(teammates)
print(answers)

['gpt-4o-mini']
['Balancing the ethical implications of AI advancements with innovation and economic growth, while ensuring equitable access and minimizing potential harms, requires a multifaceted approach. Here are several strategies that societies can adopt:\n\n1. **Framework for Ethical AI Development**:\n   - Establish clear ethical guidelines and frameworks for the development and deployment of AI technologies. This includes principles such as fairness, transparency, accountability, and respect for privacy.\n   - Encourage the involvement of ethicists, social scientists, and diverse stakeholders in the design and implementation of AI systems to ensure a variety of perspectives and values are considered.\n\n2. **Inclusive Stakeholder Engagement**:\n   - Create forums for public dialogue that include voices from various demographics, including marginalized communities, to ensure that AI advancements meet the needs of all citizens.\n   - Involve interdisciplinary teams in AI developm

In [10]:
# It's nice to know how to use "zip"
for teammate, answer in zip(teammates, answers):
    print(f"Teammate: {teammate}\n\n{answer}")

Teammate: gpt-4o-mini

Balancing the ethical implications of AI advancements with innovation and economic growth, while ensuring equitable access and minimizing potential harms, requires a multifaceted approach. Here are several strategies that societies can adopt:

1. **Framework for Ethical AI Development**:
   - Establish clear ethical guidelines and frameworks for the development and deployment of AI technologies. This includes principles such as fairness, transparency, accountability, and respect for privacy.
   - Encourage the involvement of ethicists, social scientists, and diverse stakeholders in the design and implementation of AI systems to ensure a variety of perspectives and values are considered.

2. **Inclusive Stakeholder Engagement**:
   - Create forums for public dialogue that include voices from various demographics, including marginalized communities, to ensure that AI advancements meet the needs of all citizens.
   - Involve interdisciplinary teams in AI development

In [23]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from teammate {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [36]:
formatter = f"""You are taking the nost interesting ideas fron  {len(teammates)} teammates.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, select the most relevant ideas and make a report, including a title, subtitles to separate sections, and quoting the LLM providing the idea.
From that, you will create a new improved answer."""

In [ ]:
print(formatter)

In [38]:
formatter_messages = [{"role": "user", "content": formatter}]

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=formatter_messages,
)
results = response.choices[0].message.content
display(Markdown(results))